# Instalar as Dependências

In [9]:
!pip install crewai
!pip install transformers
!pip install huggingface_hub
!pip install requests
!pip install yfinance

# Importar as Bibliotecas

In [10]:
import crewai
import yfinance as yf
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import json


#  Autenticação no Hugging Face

In [16]:
from google.colab import userdata

huggingface_token = userdata.get('huggingface_token')
login(token=huggingface_token)

print("Login realizado com sucesso!")


Login realizado com sucesso!


# Carregar o Modelo Llama 3.2 1B

In [17]:
print("Carregando o modelo LLaMA 3.2 1B...")

model_name = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Modelo LLaMA 3.2 1B carregado com sucesso!")

def llama_response(input_text):
    """
    Função para gerar uma resposta usando o modelo LLaMA 3.2 1B.
    """
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Carregando o modelo LLaMA 3.2 1B...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Modelo LLaMA 3.2 1B carregado com sucesso!


# Criar o Projeto CrewAI

In [ ]:
!crewai create crew stock_analysis

# Criar as Ferramentas para Dados de Ações

In [35]:
# Importando bibliotecas necessárias
import yfinance as yf

# Definindo a função para pesquisa de ações
def search_stock_data(stock_symbol):
    # Pesquisa os dados históricos da ação
    stock = yf.Ticker(stock_symbol)
    data = stock.history(period="5d")  # Últimos 5 dias de dados
    return data

# Função para analisar o desempenho da ação
def analyze_stock_performance(stock_data):
    current_price = stock_data['Close'][-1]
    average_price = stock_data['Close'].mean()

    if current_price > average_price:
        return f"A ação está em alta. Preço atual: {current_price}, Média dos últimos 5 dias: {average_price}"
    else:
        return f"A ação está em baixa. Preço atual: {current_price}, Média dos últimos 5 dias: {average_price}"

# Função para gerar o relatório
def generate_report(stock_symbol, stock_data, analysis_result):
    current_price = round(stock_data['Close'][-1], 2)  # Preço de fechamento atual
    average_price = round(stock_data['Close'].mean(), 2)  # Média dos últimos 5 dias

    report = f"🚀 Relatório Final:\n\n"
    report += f"📊 Relatório de Desempenho da Ação: {stock_symbol} 📉\n\n"
    report += f"💡 Últimos Dados de Cotação:\n"
    report += f"🗓️ Data mais recente: {stock_data.index[-1].date()}\n"
    report += f"💵 Preço Atual: {current_price} USD\n"
    report += f"📈 Média dos últimos 5 dias: {average_price} USD\n\n"
    report += f"🔍 Análise do Desempenho:\n"
    report += f"{analysis_result}\n\n"

    # Frase final personalizada com emojis
    if stock_symbol == "TSLA":
        report += "⚡ TSLA continua a surpreender com sua alta performance! Vamos aproveitar essa oportunidade! 🚗💨\n"
    elif stock_symbol == "AAPL":
        report += "🍏 AAPL está enfrentando um desafio, mas sempre é uma ação para monitorar de perto. 🧐📱\n"
    elif stock_symbol == "AMZN":
        report += "📦 AMZN tem mostrado grande estabilidade, ideal para quem busca segurança no longo prazo! 🔒💼\n"
    else:
        report += "🔍 Mantenha-se atento às flutuações do mercado e aja com cautela! 💼📊\n"

    # Resumo final
    report += "\n🔑 Resumo Final:\n"
    report += f"📉 A ação {stock_symbol} está com um desempenho de {'alta' if 'alta' in analysis_result.lower() else 'baixa'}.\n"
    report += f"📊 Preço Atual: {current_price} USD, Média dos Últimos 5 Dias: {average_price} USD\n"
    report += "🔍 Recomendação: Com base nos dados, monitore a ação de perto e faça decisões informadas. 📊"

    # Retornando o relatório gerado
    return report


In [31]:
# Definindo as ferramentas para os agentes

class StockSearchTool:
    def __init__(self):
        pass

    def search(self, stock_symbol):
        # Chama a função para pesquisar dados históricos da ação
        return search_stock_data(stock_symbol)


class StockAnalysisTool:
    def __init__(self):
        pass

    def analyze(self, stock_data):
        # Chama a função para analisar o desempenho da ação
        return analyze_stock_performance(stock_data)


class ReportGenerationTool:
    def __init__(self):
        pass

    def generate(self, stock_symbol, stock_data, analysis_result):
        # Chama a função para gerar o relatório da ação
        return generate_report(stock_symbol, stock_data, analysis_result)


# Agentes usando as ferramentas criadas

class Agent:
    def __init__(self, role, goal, backstory, tools, verbose=False):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.tools = tools
        self.verbose = verbose


Agentes criados e integrados com sucesso!


# Criando Agentes para Dados de Ações


In [32]:
agent_search = Agent(
    role="Pesquisador de Ações",
    goal="Pesquisar dados financeiros de ações",
    backstory="Este agente busca os dados históricos e financeiros de ações para análise.",
    tools=[StockSearchTool()],  # Usando a ferramenta de pesquisa
    verbose=True
)

# Criar o agente de Análise de Ação
agent_analysis = Agent(
    role="Analista de Ações",
    goal="Analisar o desempenho das ações comparando com a média dos últimos dias",
    backstory="Este agente compara o desempenho atual da ação com a média dos últimos 5 dias.",
    tools=[StockAnalysisTool()],  # Usando a ferramenta de análise
    verbose=True
)

# Criar o agente de Geração de Relatórios
agent_report = Agent(
    role="Gerador de Relatórios de Ações",
    goal="Gerar um relatório sobre o desempenho das ações",
    backstory="Este agente gera relatórios detalhados baseados nos dados e análises das ações.",
    tools=[ReportGenerationTool()],  # Usando a ferramenta de geração de relatórios
    verbose=True
)

print("Agentes criados e integrados com sucesso!")

Agentes criados e integrados com sucesso!


# Definindo o Fluxo de Trabalho

In [39]:
# Fluxo de trabalho para múltiplas ações com salvamento dos relatórios em arquivos .txt
def workflow(stock_symbols, save_to_file=True, folder_path="/content"):
    final_reports = []

    for stock_symbol in stock_symbols:
        print(f"🌟 Iniciando Fluxo de Trabalho para a Ação: {stock_symbol} 🚀")

        # Passo 1: Pesquisa - O agente pesquisa os dados financeiros da ação
        print(f"🔍 Pesquisando dados financeiros da ação {stock_symbol}...")
        stock_data = agent_search.tools[0].search(stock_symbol)  # Chama o método correto 'search'
        print(f"✅ Dados de pesquisa para {stock_symbol} foram coletados com sucesso!\n")

        # Passo 2: Análise - O agente analisa o desempenho da ação
        print(f"📊 Analisando desempenho da ação {stock_symbol}...")
        analysis_result = agent_analysis.tools[0].analyze(stock_data)
        print(f"✅ Análise de desempenho para {stock_symbol} foi realizada!\n")

        # Passo 3: Geração de Relatório - O agente gera um relatório sobre a ação
        print(f"📑 Gerando relatório para a ação {stock_symbol}...")
        report = agent_report.tools[0].generate(stock_symbol, stock_data, analysis_result)
        final_reports.append(report)
        print(f"✅ Relatório para {stock_symbol} gerado com sucesso!\n")

        # Salvar o relatório em um arquivo .txt
        if save_to_file:
            report_filename = f"{folder_path}/{stock_symbol}_report.txt"
            with open(report_filename, "w", encoding="utf-8") as file:
                file.write(report)
            print(f"✅ Relatório salvo como {report_filename}\n")

    return final_reports

# Rodando o Fluxo de Trabalho

In [37]:
# Testar o fluxo de trabalho com múltiplas ações
stock_symbols = ["TSLA", "AAPL", "AMZN"]
final_reports = workflow(stock_symbols)

# Exibir os relatórios finais gerados para todas as ações
for report in final_reports:
    print(report)
    print("\n" + "="*50 + "\n")

🌟 Iniciando Fluxo de Trabalho para a Ação: TSLA 🚀
🔍 Pesquisando dados financeiros da ação TSLA...
✅ Dados de pesquisa para TSLA foram coletados com sucesso!

📊 Analisando desempenho da ação TSLA...
✅ Análise de desempenho para TSLA foi realizada!

📑 Gerando relatório para a ação TSLA...
✅ Relatório para TSLA gerado com sucesso!

🌟 Iniciando Fluxo de Trabalho para a Ação: AAPL 🚀
🔍 Pesquisando dados financeiros da ação AAPL...
✅ Dados de pesquisa para AAPL foram coletados com sucesso!

📊 Analisando desempenho da ação AAPL...
✅ Análise de desempenho para AAPL foi realizada!

📑 Gerando relatório para a ação AAPL...
✅ Relatório para AAPL gerado com sucesso!

🌟 Iniciando Fluxo de Trabalho para a Ação: AMZN 🚀
🔍 Pesquisando dados financeiros da ação AMZN...


<ipython-input-35-35232865229e>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Close'][-1]
<ipython-input-35-35232865229e>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = round(stock_data['Close'][-1], 2)  # Preço de fechamento atual
<ipython-input-35-35232865229e>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Close'][-1]
<ipython-input-35-35232865229e>

✅ Dados de pesquisa para AMZN foram coletados com sucesso!

📊 Analisando desempenho da ação AMZN...
✅ Análise de desempenho para AMZN foi realizada!

📑 Gerando relatório para a ação AMZN...
✅ Relatório para AMZN gerado com sucesso!

🚀 Relatório Final:

📊 Relatório de Desempenho da Ação: TSLA 📉

💡 Últimos Dados de Cotação:
🗓️ Data mais recente: 2024-11-15
💵 Preço Atual: 320.72 USD
📈 Média dos últimos 5 dias: 328.13 USD

🔍 Análise do Desempenho:
A ação está em baixa. Preço atual: 320.7200012207031, Média dos últimos 5 dias: 328.12599487304686

⚡ TSLA continua a surpreender com sua alta performance! Vamos aproveitar essa oportunidade! 🚗💨

🔑 Resumo Final:
📉 A ação TSLA está com um desempenho de baixa.
📊 Preço Atual: 320.72 USD, Média dos Últimos 5 Dias: 328.13 USD
🔍 Recomendação: Com base nos dados, monitore a ação de perto e faça decisões informadas. 📊


🚀 Relatório Final:

📊 Relatório de Desempenho da Ação: AAPL 📉

💡 Últimos Dados de Cotação:
🗓️ Data mais recente: 2024-11-15
💵 Preço Atua

<ipython-input-35-35232865229e>:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Close'][-1]
<ipython-input-35-35232865229e>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = round(stock_data['Close'][-1], 2)  # Preço de fechamento atual


# Testar o Fluxo de Trabalho com o Telegram

In [ ]:
!pip install python-telegram-bot

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
!pip show python-telegram-bot


In [40]:
import os
from telegram import Bot
from google.colab import userdata

bot_token = userdata.get('bot_token')

# Função assíncrona para enviar o conteúdo do arquivo como mensagem no Telegram
async def send_report_to_telegram(bot_token, chat_id, file_path):
    """
    Envia o conteúdo do arquivo gerado para o Telegram, como uma mensagem de texto.
    """
    bot = Bot(token=bot_token)

    # Ler o conteúdo do arquivo
    with open(file_path, "r") as file:
        file_content = file.read()

    # Enviar o conteúdo do arquivo como mensagem de texto
    await bot.send_message(chat_id=chat_id, text=file_content)

# Função para enviar todos os arquivos .txt como mensagens no Telegram
async def send_all_reports_to_telegram(bot_token, chat_id, folder_path="/content"):
    """
    Envia todos os arquivos .txt no diretório especificado como mensagens de texto para o Telegram.
    """
    # Encontrar todos os arquivos .txt no diretório especificado
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    # Enviar o conteúdo de cada arquivo como mensagem
    for txt_file in txt_files:
        file_path = os.path.join(folder_path, txt_file)
        await send_report_to_telegram(bot_token, chat_id, file_path)

# Agora, como o código é assíncrono, você deve chamar a função da seguinte maneira
await send_all_reports_to_telegram(bot_token, 'IDCANAL')
